In [ ]:
#@title Default title text
!pip install -U keras-tuner
!pip install hyperopt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Prediction Code#
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation , Dropout
from tensorflow.keras import layers
import time
import pickle
from keras.utils import np_utils
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_height = 100
img_width = 100

train_dir='/content/drive/MyDrive/Colab Notebooks/train-test-80-20-Vol2/train-test-80-20/Train-Test-80-20/Train'
test_dir='/content/drive/MyDrive/Colab Notebooks/train-test-80-20-Vol2/train-test-80-20/Train-Test-80-20/Test'
validation_dir='/content/drive/MyDrive/Colab Notebooks/train-test-80-20-Vol2/train-test-80-20/Train-Test-80-20/Test'

labels = ['Hasta', 'Saglikli']
img_size = 100

def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)


train=get_data(train_dir)
test=get_data(test_dir)
valid=get_data(validation_dir)


x_train=[]
y_train=[]

x_test=[]
y_test=[]

x_val=[]
y_val=[]


for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in valid:
    x_val.append(feature)
    y_val.append(label)


x_train = np.array(x_train) /255.0
x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)
y_train=np_utils.to_categorical(y_train) 

x_test = np.array(x_test) /255.0
x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)
y_test=np_utils.to_categorical(y_test) 

x_val = np.array(x_val) /255.0
x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)
y_val=np_utils.to_categorical(y_val) 

from sklearn.utils import shuffle
x_train,y_train=shuffle(x_train,y_train)
x_test,y_test=shuffle(x_test,y_test)
x_val,y_val=shuffle(x_val,y_val)

train_datagen=ImageDataGenerator(brightness_range=(0.1,1.0))
train_datagen.fit(x_train)

model=keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(128,(3,3),input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=5))
model.add(Activation('relu'))
model.add(Conv2D(filters=16,kernel_size=1))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(2,activation='softmax'))


model.compile(loss="binary_crossentropy",
                  optimizer=keras.optimizers.Adam(learning_rate=1e-5,beta_1=0.9,
                                 beta_2=0.9)
                  ,metrics=["accuracy"])
    
history = model.fit(x_train,y_train,32,200)#,validation_data=(x_test,y_test))
    
from sklearn.metrics import classification_report
y_test=np.argmax(y_test, axis=1)
predictions = model.predict_classes(x_test)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_test, predictions, target_names = ['hasta (Class 0)','saglikli (Class 1)']))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
7/7 [==============================] - 8s 1s/step - loss: 0.7002 - accuracy: 0.4866
Epoch 2/200
7/7 [==============================] - 7s 1s/step - loss: 0.6772 - accuracy: 0.5736
Epoch 3/200
7/7 [==============================] - 7s 1s/step - loss: 0.6803 - accuracy: 0.5562
Epoch 4/200
7/7 [==============================] - 7s 1s/step - loss: 0.6644 - accuracy: 0.7486
Epoch 5/200
7/7 [==============================] - 7s 1s/step - loss: 0.6435 - accuracy: 0.5674
Epoch 6/200
7/7 [==============================] - 7s 1s/step - loss: 0.6237 - accuracy: 0.6028
Epoch 7/200
7/7 [==============================] - 7s 1s/step - loss: 0.6074 - accuracy: 0.7033
Epoch 8/200
7/7 [==============================] - 7s 1s/step - loss: 0.5975 - accuracy: 0.7894
Epoch 9/200
7/7 [==============================] - 7s 1s/step - loss: 0.5715 - accuracy: 0.7668
Epoch 10/200
7/7 [==============================] - 7s 1s/step - loss: 0.5601 - accuracy: 0.8275
Epoch 11/200
7/7 [=====================

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


                    precision    recall  f1-score   support

   hasta (Class 0)       0.90      0.96      0.93        28
saglikli (Class 1)       0.96      0.89      0.93        28

          accuracy                           0.93        56
         macro avg       0.93      0.93      0.93        56
      weighted avg       0.93      0.93      0.93        56



In [ ]:
#####MAIN SEARCH BAYESIAN##############
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation , Dropout
from tensorflow.keras import layers
from kerastuner import tuners
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle
from keras.utils import np_utils
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import kerastuner 
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

LOG_DIR = f"{int(time.time())}"

tensorboard = TensorBoard(log_dir=LOG_DIR)

batch_size = 32
img_height = 150
img_width = 150

train_dir='/content/drive/MyDrive/Colab Notebooks/train-test-60-15-25/train-test-80-20/Train-Test-80-20/Train'
test_dir='/content/drive/MyDrive/Colab Notebooks/train-test-60-15-25/train-test-80-20/Train-Test-80-20/Test'
validation_dir='/content/drive/MyDrive/Colab Notebooks/train-test-60-15-25/train-test-80-20/Train-Test-80-20/Valid'

labels = ['Hasta', 'Saglikli']
img_size = 150

def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)


train=get_data(train_dir)
test=get_data(test_dir)
valid=get_data(validation_dir)


x_train=[]
y_train=[]

x_test=[]
y_test=[]

x_val=[]
y_val=[]


for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in valid:
    x_val.append(feature)
    y_val.append(label)


x_train = np.array(x_train) /255.0
x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)
y_train=np_utils.to_categorical(y_train) 

x_test = np.array(x_test) /255.0
x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)
y_test=np_utils.to_categorical(y_test) 

x_val = np.array(x_val) /255.0
x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)
y_val=np_utils.to_categorical(y_val) 

from sklearn.utils import shuffle
x_train,y_train=shuffle(x_train,y_train)
x_test,y_test=shuffle(x_test,y_test)
x_val,y_val=shuffle(x_val,y_val)

train_datagen=ImageDataGenerator(brightness_range=(0.1,1.0))
train_datagen.fit(x_train)
def build_model(hp):   
    model = keras.models.Sequential()

    model.add(Conv2D(hp.Int('input_conv_units',min_value=32,max_value=512,step=32),
                     kernel_size=3,input_shape=x_train.shape[1:]))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))                
    for i in range(hp.Int('n_of_conv_layers',1,5)):
        model.add(Conv2D(hp.Int(f'Conv_{i}_Size',min_value=32,max_value=512,step=32),
                  kernel_size=hp.Choice('kernel_size_of_conv'+str(i),[1,3,5,7])))
        model.add(Activation('relu'))           
    
    model.add(Flatten())
    
    for j in range(hp.Int('n_of_dense_layers',1,5)):
        model.add(Dense(hp.Int(f'Dense_{j}_Size',min_value=8,max_value=256,step=8)))
        model.add(Activation('relu'))
        model.add(Dropout(hp.Float('Dropout_rate'+str(j), min_value=0.0,max_value=0.3,step=0.00000001)))
    
    
    model.add(Dense(2))
    model.add(Activation("softmax"))

    model.compile(loss="binary_crossentropy",
                  optimizer=keras.optimizers.Adam(learning_rate=hp.Float('Learning_Rate',min_value=1e-5,max_value=1e-2,step=0.00000001),
                                 beta_1=hp.Float('beta1',min_value=0.9,max_value=0.999999,step=0.00000001),
                                 beta_2=hp.Float('beta2',min_value=0.9,max_value=0.999999,step=0.00000001)),
                  metrics=["accuracy"])

    return model

tuner = kerastuner.tuners.bayesian.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,  
    executions_per_trial=1,
    directory=LOG_DIR)

tuner.search_space_summary()


tuner.search(x=x_train,
             y=y_train,
             epochs=30,
             batch_size=32,
             callbacks=[tensorboard],
             verbose=1,
             validation_data=(x_val, y_val))

tuner.results_summary() 


with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)

In [ ]:
#####Transfer SEARCH BAYESIAN##############
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation , Dropout
from tensorflow.keras import layers
from kerastuner import tuners
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle
from keras.utils import np_utils
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import kerastuner 
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

LOG_DIR = f"{int(time.time())}"

tensorboard = TensorBoard(log_dir=LOG_DIR)

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPooling2D, Flatten , Activation,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
from hyperopt import tpe, fmin, hp
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
batch_size = 64
img_height = 160
img_width = 160

train_dir='/content/drive/MyDrive/Colab Notebooks/Train-Test-Valid-60-20-20/Train'
test_dir='/content/drive/MyDrive/Colab Notebooks/Train-Test-Valid-60-20-20/Test'
validation_dir='/content/drive/MyDrive/Colab Notebooks/Train-Test-Valid-60-20-20/Valid'

labels = ['Hasta', 'Saglikli']
img_size = 160

def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)



train=get_data(train_dir)
test=get_data(test_dir)
valid=get_data(validation_dir)


x_train=[]
y_train=[]

x_test=[]
y_test=[]

x_val=[]
y_val=[]


for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in valid:
    x_val.append(feature)
    y_val.append(label)



y_test = np.array(y_test)
y_train=np.array(y_train)
y_val=np.array(y_val)

x_test=np.array(x_test)
x_train=np.array(x_train)
x_val=np.array(x_val)


from sklearn.utils import shuffle
x_train,y_train=shuffle(x_train,y_train)
x_test,y_test=shuffle(x_test,y_test)
x_val,y_val=shuffle(x_val,y_val)

def build_model(hp):   
    data_augmentation = tf.keras.Sequential([

    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1,1.0)
    ])

    base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height,img_width) + (3,),
                                               include_top=False,
                                               weights='imagenet')
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(img_height,img_width) + (3,))
    x = data_augmentation(inputs)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    x = base_model(x, training=False)
    x = tf.keras.layers.Flatten()(x)
    for j in range(hp.Int('n_of_dense_layers',1,5)):
        x = tf.keras.layers.Dense(hp.Int(f'Dense_{j}_Size',min_value=8,max_value=256,step=8),activation='relu')(x)
        x = tf.keras.layers.Dropout(hp.Float('Dropout_rate'+str(j), min_value=0.0,max_value=0.5,step=0.00000001))(x)
    outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(loss="binary_crossentropy",
                  optimizer=keras.optimizers.Adam(learning_rate=hp.Float('Learning_Rate',min_value=1e-5,max_value=1e-2,step=0.00000001),
                                 beta_1=hp.Float('beta1',min_value=0.9,max_value=0.999999,step=0.00000001),
                                 beta_2=hp.Float('beta2',min_value=0.9,max_value=0.999999,step=0.00000001)),
                  metrics=["accuracy"])

    return model

tuner = kerastuner.tuners.bayesian.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,  
    executions_per_trial=1,
    directory=LOG_DIR)

tuner.search_space_summary()


tuner.search(x=x_train,
             y=y_train,
             epochs=30,
             batch_size=64,
             callbacks=[tensorboard],
             verbose=1,
             validation_data=(x_val, y_val))

tuner.results_summary() 


with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)

In [ ]:
tuner = pickle.load(open("tuner_1617716020.pkl","rb"))
tuner.get_best_hyperparameters()[0].values
tuner.get_best_models()[0].summary()

In [ ]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);